In the problem http://puzzlor.com/2008-08_MarkovsPrison.html, we are asked to find an escape route for a prisoner that minimizes their probability of being caught.  Since the guards move from room to room with given probabilities independent of their historical states (and because the word is in the title of the puzzle) we can use a Markov chain to determine the safest path for the prisoner to take.  The guards have been moving through the prison for years, which means that the Markov chain will be well-mixed, so we can make use of the long term transition probabilities that correspond to the left eigenvectors of the transition matrix.   Since there are 16 rooms, we have 16 possible states for each guard, and our transition matrices will be 16 x 16.  We can either find the left eigenvector for each matrix associated to the eigenvalue \lambda = 1, or simply take a large power of each matrix to approximate the limiting distribution.  Once we have our limiting distributions, we need to find the path from 1 to 16 that constitutes the smallest probability of getting caught.

In [49]:
import numpy as np
from numpy import matlib
import scipy.sparse as sp
np.set_printoptions(precision = 2)

In [3]:
# create the transition matrix for the first guard
G1 = np.array([[.4, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, .2, .2, 0, 0,.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, .2, .2, 0, 0,.4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, .4, 0, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, 0, 0, 0, .2, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0, 0, 0, 0],
               [0, 0, 0, .2, 0, 0, .2, .2, 0, 0, 0, .4, 0, 0, 0, 0],
               [0, 0, 0, 0, .2, 0, 0, 0, .2, .2, 0, 0, .4, 0, 0, 0],
               [0, 0, 0, 0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0, 0],
               [0, 0, 0, 0, 0, 0, .2, 0, 0, .2, 0, .2, 0, 0, .4, 0],
               [0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .2, 0, 0, 0, .4],
               [0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, 0, .6, .2, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .4, .2, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .4, .2],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, .2, 0, 0, .2, .6]])

In [4]:
G1.shape

(16, 16)

In [5]:
# check that we are indeed a stochastic matrix
np.sum(G1,axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [6]:
G2 = np.array([[.6, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [.2, .4, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, .2, .4, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, .2, .7, 0, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0, 0],
               [.4, 0, 0, 0, .2, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0, 0],
               [0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0, 0, 0, 0, 0],
               [0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0, 0, 0, 0],
               [0, 0, 0, .4, 0, 0, .2, .3, 0, 0, 0, .1, 0, 0, 0, 0],
               [0, 0, 0, 0, .4, 0, 0, 0, .2, .3, 0, 0, .1, 0, 0, 0],
               [0, 0, 0, 0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0, 0],
               [0, 0, 0, 0, 0, 0, .4, 0, 0, .2, 0, .3, 0, 0, .1, 0],
               [0, 0, 0, 0, 0, 0, 0, .4, 0, 0, .2, .3, 0, 0, 0, .1],
               [0, 0, 0, 0, 0, 0, 0, 0, .4, 0, 0, 0, .3, .3, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, .4, 0, 0, .2, .1, .3, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0,  0, .4, 0, 0, .2, .1, .3],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, .4, 0, 0, .2, .4]])

In [8]:
np.sum(G2,axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [30]:
# we can approximate by taking a large power of the matrix
G1_steady_state = np.linalg.matrix_power(G1,10000)[0,:]
#print sum(G1_steady_state)


In [31]:
G2_steady_state = np.linalg.matrix_power(G2,10000)[0,:]
#print sum(G2_steady_state)

now that we have our limiting distributions, we can create a mask using the original costs (to prevent us from hopping between non adjacent rooms) and use Dijkstra's method to find the shortest path from 1 to 16, with the edge costs being the probability of a guard being in the rooms

In [32]:
captureProbabilities = G1_steady_state+G2_steady_state-G1_steady_state*G2_steady_state
print captureProbabilities

[ 0.11  0.15  0.22  0.32  0.06  0.07  0.08  0.11  0.07  0.07  0.08  0.08
  0.13  0.14  0.14  0.14]


In [52]:
# use the transition matrix G1 as a mask to make the correct weight matrix for Dijkstra
dijkstra_matrix = (G1!=0)*matlib.repmat(captureProbabilities,16,1)

In [54]:
dist_matrix, pred = sp.csgraph.dijkstra(csgraph=dijkstra_matrix, return_predecessors = True)

In [55]:
next_node = 15
nodes = []
while next_node != 0:
    nodes.append(next_node)
    next_node = pred[0][next_node]

In [56]:
print pred

[[-9999     0     1     7     0     4     5     6     4     5     9    10
      8     9    10    11]
 [    1 -9999     1     2     5     1     5     6     4     5     9    10
      8     9    10    11]
 [    1     2 -9999     2     5     6     2     6     4     5     6    10
      8     9    10    11]
 [    4     2     3 -9999     5     6     7     3     4     5     6     7
      8     9    10    11]
 [    4     5     6     7 -9999     4     5     6     4     5     9    10
      8     9    10    11]
 [    4     5     6     7     5 -9999     5     6     4     5     9    10
      8     9    10    11]
 [    4     5     6     7     5     6 -9999     6     4     5     6    10
      8     9    10    11]
 [    4     5     6     7     5     6     7 -9999     4     5     6     7
      8     9    10    11]
 [    4     5     6     7     8     4     5     6 -9999     8     9    10
      8     9    10    11]
 [    4     5     6     7     5     9     5     6     9 -9999     9    10
      8     9    

In [57]:
nodes.reverse()

In [58]:
for node in nodes:
    print node+1

5
6
10
11
12
16
